In [4]:
from planet4.get_data import get_image_id

In [6]:
df = get_image_id('APF0001m7c')

In [44]:
df.head()

,classification_id,created_at,image_id,image_name,image_url,user_name,marking,x_tile,y_tile,acquisition_date,...,x,y,image_x,image_y,radius_1,radius_2,distance,angle,spread,version
10979012,52a6145f9e32600583001a85,2013-12-09 19:05:03,APF0001m7c,PSP_003453_0945,http://www.planetfour.org/subjects/standard/51...,Mildredo,blotch,5,3,2007-04-22,...,740.5,51,3700.5,1147,27.018512,20.263884,NaN,87.878904,0,1
10979013,52a6145f9e32600583001a85,2013-12-09 19:05:03,APF0001m7c,PSP_003453_0945,http://www.planetfour.org/subjects/standard/51...,Mildredo,blotch,5,3,2007-04-22,...,700.5,638,3660.5,1734,50.695167,38.021376,NaN,-14.858614,0,1
11756671,539b0d3fa5ae133007000218,2014-06-13 14:39:59,APF0001m7c,PSP_003453_0945,http://www.planetfour.org/subjects/standard/51...,Djoga,blotch,5,3,2007-04-22,...,160.5,237,3120.5,1333,63.063460,47.297595,NaN,62.622297,0,1
11756672,539b0d3fa5ae133007000218,2014-06-13 14:39:59,APF0001m7c,PSP_003453_0945,http://www.planetfour.org/subjects/standard/51...,Djoga,blotch,5,3,2007-04-22,...,743.5,55,3703.5,1151,22.203603,16.652702,NaN,97.765166,0,1
11756673,539b0d3fa5ae133007000218,2014-06-13 14:39:59,APF0001m7c,PSP_003453_0945,http://www.planetfour.org/subjects/standard/51...,Djoga,blotch,5,3,2007-04-22,...,705.5,634,3665.5,1730,34.885527,26.164145,NaN,62.700428,0,1


In [50]:
cols = 'user_name classification_id created_at image_id'.split()

In [52]:
from planet4 import helper_functions as hf

In [97]:
newdf = df.set_index('created_at').sort_index()

In [115]:
g = df.groupby(['user_name','created_at'])

In [118]:
g.first?

In [77]:
def process_group(g):
    user_data.sort_index(inplace=True)
    return user_data.f

In [122]:
df.created_at.describe()

count                      68
unique                     29
top       2014-10-07 13:59:04
freq                        6
first     2013-03-20 20:20:41
last      2014-11-03 17:04:04
Name: created_at, dtype: object

In [128]:
df[df.created_at == df.created_at.min()]

,classification_id,created_at,image_id,image_name,image_url,user_name,marking,x_tile,y_tile,acquisition_date,...,x,y,image_x,image_y,radius_1,radius_2,distance,angle,spread,version
9506878,514a1a16a202b404ca007f6f,2013-03-20 20:20:41,APF0001m7c,PSP_003453_0945,http://www.planetfour.org/subjects/standard/51...,MY LUCKY STARS,none,5,3,2007-04-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [ ]:
df.groupby['user_name']

In [165]:
def process_user_group(g):
    c_id = g.sort('created_at').classification_id.iloc[0]
    return g[g.classification_id == c_id]

In [139]:
filtered_data = df.groupby(['user_name']).apply(process_user_group)

In [189]:
filtered_data.columns

Index(['classification_id', 'created_at', 'image_id', 'image_name', 'image_url', 'user_name', 'marking', 'x_tile', 'y_tile', 'acquisition_date', 'local_mars_time', 'x', 'y', 'image_x', 'image_y', 'radius_1', 'radius_2', 'distance', 'angle', 'spread', 'version'], dtype='object')

In [26]:
from planet4.get_data import get_image_name

In [27]:
dfimg = get_image_name(df.image_name.iloc[0])

In [185]:
%%timeit
container = []
for image_id in dfimg.image_id.unique():
    container.append(dfimg[dfimg.image_id == image_id].groupby('user_name').apply(process_user_group))
filtered_df = pd.concat(container).reset_index(drop=True)

1 loops, best of 3: 22.1 s per loop


In [187]:
filtered_df = dfimg.groupby(['image_id','user_name']).apply(process_user_group).reset_index(drop=True)

In [190]:
pd.read_csv?

In [149]:
dfimg.groupby('image_id').apply(hf.classification_counts_per_user).nlargest(2)

image_id    user_name
APF0001m83  mjester      7
APF0001lx1  cmoran3      6
Name: classification_id, dtype: int64

In [151]:
hf.classification_counts_per_user(dfimg[dfimg.image_id=='APF0001m83'])

user_name
davegrimble1                                      1
not-logged-in-9e5677f1a606605a01beb41b63ecc0a1    1
cornmill                                          1
willOS                                            1
angi60                                            1
Mike191919                                        1
emma.rance                                        1
doug67                                            1
pknut53                                           1
not-logged-in-3b5235c7515dadec19f60d0923281671    1
drigo                                             1
Quincy Elizabeth                                  1
servewhonor                                       1
not-logged-in-bbe7a773d210a82bdcde038cb89c5927    1
not-logged-in-61f85ec6e64b6a9fa8a1329848383c07    1
not-logged-in-55f6cac76d260fbc76bff23b576c31e5    1
Alebon                                            1
Rachel05                                          1
mjester                                           7
Na